<a href="https://colab.research.google.com/github/WhiteFox-Lugh/Pratice-TextAnalysis/blob/master/exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 情報システム論実習　テキスト分析　課題1

逐次実行していけば所定の処理ができるようにしてあります。
（ただし、外部テキストファイルなどは適切な場所に配置してください）

# 0. 前処理

In [81]:
!pip install nltk
!pip install gensim

In [82]:
import nltk
import numpy as np
import pandas as pd
import re
from gensim import models
from gensim.models.doc2vec import LabeledSentence
from google.colab import drive
from nltk.corpus import wordnet as wn
from nltk.metrics import jaccard_distance
from sklearn.feature_extraction.text import TfidfTransformer

In [83]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

text の引用元

全て最初の概要部分です（注釈番号は抜いています）

* text1.txt  : https://en.wikipedia.org/wiki/Canis
* text2.txt  : https://en.wikipedia.org/wiki/Wolf
* text3.txt  : https://en.wikipedia.org/wiki/Coyote
* text4.txt  : https://en.wikipedia.org/wiki/African_golden_wolf
* text5.txt  : https://en.wikipedia.org/wiki/Dhole
* text6.txt  : https://en.wikipedia.org/wiki/African_wild_dog
* text7.txt  : https://en.wikipedia.org/wiki/Red_fox
* text8.txt  : https://en.wikipedia.org/wiki/Arctic_fox
* text9.txt  : https://en.wikipedia.org/wiki/Fennec_fox
* text10.txt : https://en.wikipedia.org/wiki/Kitsune

In [84]:
# drive のマウント（colaboratory 用）
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
# テキスト読み込み
# テキストデータは適切なところに配置してください
text_name = ['canis', 'wolf', 'coyote', 'african_golden_wolf', 'dhole', 'african_wild_dog', 'red_fox', 'arctic_fox', 'fennec_fox', 'kitsune']
text_correction = {}
for i in range(len(text_name)):
  txt = f = open('drive/My Drive/Colab Notebooks/text' + str(i+1) + '.txt')
  data = f.read()
  name = text_name[i]
  text_correction[name] = data

In [86]:
# 読み込み確認
for txt in text_correction.values():
  print(txt)

Canis is a genus of the Caninae containing multiple extant species, such as wolves, dogs, coyotes and jackals. Species of this genus are distinguished by their moderate to large size, their massive, well-developed skulls and dentition, long legs, and comparatively short ears and tails.
The wolf (Canis lupus), also known as the gray wolf or grey wolf, is a large canine native to Eurasia and North America. More than thirty subspecies of Canis lupus have been recognized, and gray wolves, as colloquially understood, comprise non-domestic/feral subspecies. The wolf is the largest extant member of Canidae, males averaging 40 kg (88 lb) and females 37 kg (82 lb). Wolves measure 105–160 cm (41–63 in) in length and 80–85 cm (31–33 in) at shoulder height. The wolf is also distinguished from other Canis species by its less pointed ears and muzzle, as well as a shorter torso and a longer tail. The wolf is nonetheless related closely enough to smaller Canis species, such as the coyote and the golde

In [87]:
# preprocessing
# tokenize
text_tokenized = {}

def tokenize_text(text):
  text = re.sub('[.,()\[\]]', '', text)
  ret = text.split()
  return ret

for i in range(len(text_name)):
  name = text_name[i]
  txt = text_correction[name]
  text_tokenized[name] = tokenize_text(txt)

In [88]:
# tokenize 確認
for txt in text_tokenized.values():
  print(txt)

['Canis', 'is', 'a', 'genus', 'of', 'the', 'Caninae', 'containing', 'multiple', 'extant', 'species', 'such', 'as', 'wolves', 'dogs', 'coyotes', 'and', 'jackals', 'Species', 'of', 'this', 'genus', 'are', 'distinguished', 'by', 'their', 'moderate', 'to', 'large', 'size', 'their', 'massive', 'well-developed', 'skulls', 'and', 'dentition', 'long', 'legs', 'and', 'comparatively', 'short', 'ears', 'and', 'tails']
['The', 'wolf', 'Canis', 'lupus', 'also', 'known', 'as', 'the', 'gray', 'wolf', 'or', 'grey', 'wolf', 'is', 'a', 'large', 'canine', 'native', 'to', 'Eurasia', 'and', 'North', 'America', 'More', 'than', 'thirty', 'subspecies', 'of', 'Canis', 'lupus', 'have', 'been', 'recognized', 'and', 'gray', 'wolves', 'as', 'colloquially', 'understood', 'comprise', 'non-domestic/feral', 'subspecies', 'The', 'wolf', 'is', 'the', 'largest', 'extant', 'member', 'of', 'Canidae', 'males', 'averaging', '40', 'kg', '88', 'lb', 'and', 'females', '37', 'kg', '82', 'lb', 'Wolves', 'measure', '105–160', 'cm'

In [89]:
# lemmatize
def lemmatize_word(word_set):
  ret = []
  for word in word_set:
    word=word.lower()
    lemma = wn.morphy(word)
    if lemma is None:
      ret.append(word)
    else:
      ret.append(lemma)
  return ret

In [90]:
preprocessed_docs = {}

for i in range(len(text_name)):
  name = text_name[i]
  word_set = text_tokenized[name]
  preprocessed_docs[name] = lemmatize_word(word_set)

In [91]:
# preprocessing の check
for doc in preprocessed_docs.values():
  print(doc)

['canis', 'be', 'a', 'genus', 'of', 'the', 'caninae', 'contain', 'multiple', 'extant', 'species', 'such', 'as', 'wolf', 'dog', 'coyote', 'and', 'jackal', 'species', 'of', 'this', 'genus', 'are', 'distinguish', 'by', 'their', 'moderate', 'to', 'large', 'size', 'their', 'massive', 'well-developed', 'skull', 'and', 'dentition', 'long', 'legs', 'and', 'comparatively', 'short', 'ear', 'and', 'tails']
['the', 'wolf', 'canis', 'lupus', 'also', 'know', 'as', 'the', 'gray', 'wolf', 'or', 'grey', 'wolf', 'be', 'a', 'large', 'canine', 'native', 'to', 'eurasia', 'and', 'north', 'america', 'more', 'than', 'thirty', 'subspecies', 'of', 'canis', 'lupus', 'have', 'be', 'recognize', 'and', 'gray', 'wolf', 'as', 'colloquially', 'understand', 'comprise', 'non-domestic/feral', 'subspecies', 'the', 'wolf', 'be', 'the', 'large', 'extant', 'member', 'of', 'canidae', 'male', 'average', '40', 'kg', '88', 'lb', 'and', 'female', '37', 'kg', '82', 'lb', 'wolf', 'measure', '105–160', 'cm', '41–63', 'in', 'in', 'le

In [92]:
# 単語の set を構築する
docs_words_set = {}

for i in range(len(text_name)):
  name = text_name[i]
  words = preprocessed_docs[name]
  docs_words_set[name] = set(words)

In [93]:
# set の check
for word_set in docs_words_set.values():
  print(word_set)

{'skull', 'legs', 'multiple', 'dentition', 'canis', 'moderate', 'species', 'short', 'extant', 'such', 'as', 'a', 'comparatively', 'large', 'of', 'tails', 'massive', 'caninae', 'wolf', 'are', 'long', 'and', 'ear', 'well-developed', 'genus', 'contain', 'this', 'their', 'to', 'distinguish', 'by', 'size', 'coyote', 'jackal', 'be', 'the', 'dog'}
{'have', 'nonetheless', 'shoulder', 'other', 'gray', 'extant', 'region', 'them', 'le', 'mottle', 'wolf', 'at', 'fertile', 'although', 'ear', 'understand', 'recognize', 'it', 'comprise', 'non-domestic/feral', '80–85', 'smaller', 'fur', 'band', 'the', '40', 'male', 'canis', 'species', 'native', 'lupus', 'closely', 'from', '88', 'black', 'as', 'kg', 'of', 'also', '82', 'all', 'white', 'longer', 'measure', 'nearly', 'coyote', '37', 'be', 'member', 'produce', 'point', '31–33', 'enough', 'subspecies', 'golden', 'america', 'such', 'thirty', 'arctic', 'muzzle', 'a', 'may', 'well', 'grey', 'torso', 'and', 'canine', 'length', 'relate', 'to', 'distinguish', 'c

In [94]:
# bag of words でベクトル化
def bow_vectorizer(docs):
  word2id = {}
  for doc in docs.values():
    for w in doc:
      if w not in word2id:
        word2id[w] = len(word2id)
        
  result_list = []
  for doc in docs.values():
    doc_vec = [0] * len(word2id)
    for w in doc:
      doc_vec[word2id[w]] += 1
    result_list.append(doc_vec)
  return result_list, word2id

bow_vec_doc, word2id_doc = bow_vectorizer(preprocessed_docs)

In [95]:
# bow の check
for i in range(len(text_name)):
  name = text_name[i]
  vector = bow_vec_doc[i]
  print("vector(" + name + ") -> " + str(vector))

vector(canis) -> [1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [96]:
word2id_doc.items()

dict_items([('canis', 0), ('be', 1), ('a', 2), ('genus', 3), ('of', 4), ('the', 5), ('caninae', 6), ('contain', 7), ('multiple', 8), ('extant', 9), ('species', 10), ('such', 11), ('as', 12), ('wolf', 13), ('dog', 14), ('coyote', 15), ('and', 16), ('jackal', 17), ('this', 18), ('are', 19), ('distinguish', 20), ('by', 21), ('their', 22), ('moderate', 23), ('to', 24), ('large', 25), ('size', 26), ('massive', 27), ('well-developed', 28), ('skull', 29), ('dentition', 30), ('long', 31), ('legs', 32), ('comparatively', 33), ('short', 34), ('ear', 35), ('tails', 36), ('lupus', 37), ('also', 38), ('know', 39), ('gray', 40), ('or', 41), ('grey', 42), ('canine', 43), ('native', 44), ('eurasia', 45), ('north', 46), ('america', 47), ('more', 48), ('than', 49), ('thirty', 50), ('subspecies', 51), ('have', 52), ('recognize', 53), ('colloquially', 54), ('understand', 55), ('comprise', 56), ('non-domestic/feral', 57), ('member', 58), ('canidae', 59), ('male', 60), ('average', 61), ('40', 62), ('kg', 63

# 1. ドキュメントの類似度の計算

## 1-1. 単語の集合(set)ベース

### Jaccard 係数

In [97]:
# Jaccard 係数
jaccard_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    set_i = docs_words_set[name_i]
    set_j = docs_words_set[name_j]
    jaccard = 1 - jaccard_distance(set_i, set_j)
    name_ij = "Jaccard(" + name_i + ", " + name_j + ")"
    jaccard_res[name_ij] = jaccard

In [98]:
# Jaccard 係数の降順に出力
jaccard_sim_sorted = sorted(jaccard_res.items(), key = lambda x : -x[1])
for key, value in jaccard_sim_sorted:
  print(key + " -> " + str(value))

Jaccard(wolf, coyote) -> 0.2295081967213115
Jaccard(coyote, dhole) -> 0.20560747663551404
Jaccard(african_golden_wolf, african_wild_dog) -> 0.18309859154929575
Jaccard(coyote, african_golden_wolf) -> 0.17886178861788615
Jaccard(canis, african_wild_dog) -> 0.17525773195876293
Jaccard(coyote, african_wild_dog) -> 0.1696428571428571
Jaccard(canis, wolf) -> 0.16666666666666663
Jaccard(canis, coyote) -> 0.16666666666666663
Jaccard(wolf, african_wild_dog) -> 0.1610738255033557
Jaccard(arctic_fox, fennec_fox) -> 0.15966386554621848
Jaccard(african_golden_wolf, red_fox) -> 0.1578947368421053
Jaccard(african_wild_dog, red_fox) -> 0.15702479338842978
Jaccard(dhole, african_wild_dog) -> 0.1515151515151515
Jaccard(wolf, arctic_fox) -> 0.1476510067114094
Jaccard(coyote, red_fox) -> 0.13592233009708743
Jaccard(red_fox, fennec_fox) -> 0.1351351351351351
Jaccard(african_wild_dog, arctic_fox) -> 0.13432835820895528
Jaccard(red_fox, arctic_fox) -> 0.1311475409836066
Jaccard(wolf, african_golden_wolf) ->

### Dice 係数

In [99]:
# Dice 係数
def dice_sim(set_a, set_b):
  num_intersect = len(set.intersection(set_a, set_b))
  s = len(set_a) + len(set_b)
  ret = (2 * num_intersect / s) if s != 0 else 1.0
  return ret

In [100]:
# Dice 係数計算
dice_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    set_i = docs_words_set[name_i]
    set_j = docs_words_set[name_j]
    dice = dice_sim(set_i, set_j)
    name_ij = "Dice(" + name_i + ", " + name_j + ")"
    dice_res[name_ij] = dice

In [101]:
# Dice 係数の降順に出力
dice_sim_sorted = sorted(dice_res.items(), key = lambda x : -x[1])
for key, value in dice_sim_sorted:
  print(key + " -> " + str(value))

Dice(wolf, coyote) -> 0.37333333333333335
Dice(coyote, dhole) -> 0.34108527131782945
Dice(african_golden_wolf, african_wild_dog) -> 0.30952380952380953
Dice(coyote, african_golden_wolf) -> 0.30344827586206896
Dice(canis, african_wild_dog) -> 0.2982456140350877
Dice(coyote, african_wild_dog) -> 0.2900763358778626
Dice(canis, wolf) -> 0.2857142857142857
Dice(canis, coyote) -> 0.2857142857142857
Dice(wolf, african_wild_dog) -> 0.2774566473988439
Dice(arctic_fox, fennec_fox) -> 0.2753623188405797
Dice(african_golden_wolf, red_fox) -> 0.2727272727272727
Dice(african_wild_dog, red_fox) -> 0.2714285714285714
Dice(dhole, african_wild_dog) -> 0.2631578947368421
Dice(wolf, arctic_fox) -> 0.2573099415204678
Dice(coyote, red_fox) -> 0.23931623931623933
Dice(red_fox, fennec_fox) -> 0.23809523809523808
Dice(african_wild_dog, arctic_fox) -> 0.23684210526315788
Dice(red_fox, arctic_fox) -> 0.2318840579710145
Dice(wolf, african_golden_wolf) -> 0.22459893048128343
Dice(dhole, red_fox) -> 0.2173913043478

### Szymkiewicz-Simpson 係数

In [102]:
# Szymkiewicz-Simpson 係数
def simpson_sim(set_a, set_b):
  num_intersect = len(set.intersection(set_a, set_b))
  m = min(len(set_a), len(set_b))
  ret = num_intersect / m if m != 0 else 1.0
  return ret

In [103]:
# Simpson 係数計算
simpson_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    set_i = docs_words_set[name_i]
    set_j = docs_words_set[name_j]
    simpson = simpson_sim(set_i, set_j)
    name_ij = "Simpson(" + name_i + ", " + name_j + ")"
    simpson_res[name_ij] = simpson

In [104]:
# Simpson 係数の降順に出力
simpson_sim_sorted = sorted(simpson_res.items(), key = lambda x : -x[1])
for key, value in simpson_sim_sorted:
  print(key + " -> " + str(value))

Simpson(wolf, coyote) -> 0.5185185185185185
Simpson(canis, wolf) -> 0.5135135135135135
Simpson(canis, african_wild_dog) -> 0.4594594594594595
Simpson(coyote, african_golden_wolf) -> 0.4074074074074074
Simpson(coyote, dhole) -> 0.4074074074074074
Simpson(coyote, african_wild_dog) -> 0.35185185185185186
Simpson(canis, coyote) -> 0.35135135135135137
Simpson(canis, african_golden_wolf) -> 0.35135135135135137
Simpson(african_golden_wolf, african_wild_dog) -> 0.33766233766233766
Simpson(african_golden_wolf, red_fox) -> 0.3333333333333333
Simpson(canis, dhole) -> 0.32432432432432434
Simpson(wolf, african_wild_dog) -> 0.3116883116883117
Simpson(african_wild_dog, red_fox) -> 0.30158730158730157
Simpson(arctic_fox, fennec_fox) -> 0.30158730158730157
Simpson(canis, kitsune) -> 0.2972972972972973
Simpson(wolf, arctic_fox) -> 0.29333333333333333
Simpson(canis, fennec_fox) -> 0.2702702702702703
Simpson(wolf, fennec_fox) -> 0.2698412698412698
Simpson(dhole, african_wild_dog) -> 0.26666666666666666
Si

## 1-2. ベクトル空間モデルにおける類似度計算

### Bag of Words と Cosine 類似度

In [105]:
# cosine 類似度
def cos_sim(v_a, v_b):
  norm2_a = np.linalg.norm(v_a)
  norm2_b = np.linalg.norm(v_b)
  in_prod = np.array(v_a).dot(np.array(v_b))
  ret = (in_prod / (norm2_a * norm2_b)) if norm2_a * norm2_b != 0 else 1.0
  return ret

In [106]:
# cosine 類似度の計算
cos_sim_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    vec_i = bow_vec_doc[i]
    vec_j = bow_vec_doc[j]
    cos = cos_sim(vec_i, vec_j)
    name_ij = "Cosine(" + name_i + ", " + name_j + ")"
    cos_sim_res[name_ij] = cos

In [107]:
# Bag of Words -> Cosine 類似度の降順に出力
cos_sim_sorted = sorted(cos_sim_res.items(), key = lambda x : -x[1])
for key, value in cos_sim_sorted:
  print(key + " -> " + str(value))

Cosine(wolf, coyote) -> 0.7575238788047001
Cosine(wolf, african_golden_wolf) -> 0.7134192534977318
Cosine(coyote, african_golden_wolf) -> 0.7029959048180241
Cosine(coyote, red_fox) -> 0.6548309229329242
Cosine(african_golden_wolf, african_wild_dog) -> 0.6528761950982114
Cosine(african_wild_dog, red_fox) -> 0.6500144747718289
Cosine(coyote, african_wild_dog) -> 0.6275751587495547
Cosine(african_golden_wolf, red_fox) -> 0.626856509815389
Cosine(red_fox, arctic_fox) -> 0.6238364887671223
Cosine(wolf, african_wild_dog) -> 0.6161167294223817
Cosine(arctic_fox, fennec_fox) -> 0.6044238113553968
Cosine(coyote, dhole) -> 0.5980168115625907
Cosine(red_fox, fennec_fox) -> 0.5938291328748517
Cosine(arctic_fox, kitsune) -> 0.5821230070167418
Cosine(african_golden_wolf, dhole) -> 0.5727066906849961
Cosine(dhole, african_wild_dog) -> 0.5632061489195213
Cosine(african_golden_wolf, kitsune) -> 0.5460352091244365
Cosine(wolf, arctic_fox) -> 0.5404335498883447
Cosine(wolf, red_fox) -> 0.5401033437084688

### TF-IDF と Cosine 類似度

In [108]:
# TF-IDF
# bow を ndarray の 2次元配列に変換
counts = np.array(bow_vec_doc)

# transformer : idf は 1 を加えて smoothing する
transformer = TfidfTransformer(smooth_idf=False)

# TF-IDF ベクトルを得る
tf_idf = transformer.fit_transform(counts)
tf_idf_array = tf_idf.toarray()
print(tf_idf_array)

[[0.093682   0.06200715 0.06854026 ... 0.         0.         0.        ]
 [0.15637417 0.1811293  0.11440753 ... 0.         0.         0.        ]
 [0.06361943 0.16843619 0.04654568 ... 0.         0.         0.        ]
 ...
 [0.         0.13110147 0.0724572  ... 0.         0.         0.        ]
 [0.         0.12129654 0.0335191  ... 0.         0.         0.        ]
 [0.         0.04256904 0.09410827 ... 0.07029394 0.07029394 0.07029394]]


In [109]:
# TF-IDF での cosine 類似度の計算
cos_sim_tfidf_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    vec_i = tf_idf_array[i]
    vec_j = tf_idf_array[j]
    cos = cos_sim(vec_i, vec_j)
    name_ij = "TF-IDF_Cosine(" + name_i + ", " + name_j + ")"
    cos_sim_tfidf_res[name_ij] = cos

In [110]:
# TF-IDF -> Cosine 類似度の降順に出力
cos_sim_tfidf_sorted = sorted(cos_sim_tfidf_res.items(), key = lambda x : -x[1])

for key, value in cos_sim_tfidf_sorted:
  print(key + " -> " + str(value))

TF-IDF_Cosine(wolf, coyote) -> 0.4430564342394856
TF-IDF_Cosine(coyote, african_golden_wolf) -> 0.3538887335465151
TF-IDF_Cosine(wolf, african_golden_wolf) -> 0.32420592239519097
TF-IDF_Cosine(african_golden_wolf, african_wild_dog) -> 0.3079228066002627
TF-IDF_Cosine(african_golden_wolf, red_fox) -> 0.2922759861622798
TF-IDF_Cosine(red_fox, arctic_fox) -> 0.2844036862071955
TF-IDF_Cosine(coyote, red_fox) -> 0.27894826591220484
TF-IDF_Cosine(african_wild_dog, red_fox) -> 0.2778114734892113
TF-IDF_Cosine(dhole, african_wild_dog) -> 0.2679042394251947
TF-IDF_Cosine(coyote, african_wild_dog) -> 0.26376271492106396
TF-IDF_Cosine(coyote, dhole) -> 0.2556777057594383
TF-IDF_Cosine(wolf, african_wild_dog) -> 0.25447864991783153
TF-IDF_Cosine(canis, wolf) -> 0.24784165701874175
TF-IDF_Cosine(arctic_fox, fennec_fox) -> 0.24732762453207352
TF-IDF_Cosine(wolf, arctic_fox) -> 0.2465887399686806
TF-IDF_Cosine(arctic_fox, kitsune) -> 0.2387686773687851
TF-IDF_Cosine(red_fox, fennec_fox) -> 0.22402462

## 1-3. 単語の分散表現とドキュメントの類似度

# 2. 考察と感想

今回はドキュメントの類似度を計算する方法として以下の文書表現手法と距離尺度を用いた。

- 文書を単語の集合で表現する
  - Jaccard 係数、Dice 係数、 Simpson 係数
- 文書をベクトルで表現する
  - Bag of Words -> Cosine 類似度
  - TF-IDF -> Cosine 類似度
- 文書を分散表現で表現する
  - word2vec, doc2vec